In [ ]:
from google.colab import files
uploaded = files.upload()

Saving id_rsa to id_rsa


In [ ]:
!mkdir -p ~/.ssh
!mv id_rsa ~/.ssh/
!chmod 600 ~/.ssh/id_rsa

In [ ]:
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts


# github.com:22 SSH-2.0-babeld-2736fb4be


In [ ]:
!ssh -T git@github.com


Hi muradhuseynov1! You've successfully authenticated, but GitHub does not provide shell access.


In [ ]:
!git remote set-url origin git@github.com:Anjaas85/authentic_vs_AI_generated_photos.git


In [1]:
!git clone https://github.com/Anjaas85/authentic_vs_AI_generated_photos


Cloning into 'authentic_vs_AI_generated_photos'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 45 (delta 2), reused 11 (delta 0), pack-reused 30 (from 1)
Receiving objects: 100% (45/45), 161.00 MiB | 13.56 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [4]:
%cd authentic_vs_AI_generated_photos

/content/authentic_vs_AI_generated_photos


In [ ]:
!git branch regFlipCheckout.pth

In [6]:
!git checkout regFlipCheckout.pth

Branch 'regFlipCheckout.pth' set up to track remote branch 'regFlipCheckout.pth' from 'origin'.
Switched to a new branch 'regFlipCheckout.pth'


In [5]:
!git pull


Already up to date.


In [8]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision.datasets import CIFAR10
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Step 1: Define paths
data_root = "./data"
train_dir = os.path.join(data_root, "train")
val_dir = os.path.join(data_root, "val")
class_dirs = ["class_0", "class_1"]

# Step 2: Create folder structure
for directory in [train_dir, val_dir]:
    for class_dir in class_dirs:
        os.makedirs(os.path.join(directory, class_dir), exist_ok=True)

# Step 3: Download CIFAR10 dataset
cifar10 = CIFAR10(root="./", download=True)
data, labels = cifar10.data, cifar10.targets

# Step 4: Simulate CIFAKE10 (first 5 classes as "authentic", next 5 as "AI-generated")
labels = [0 if label < 5 else 1 for label in labels]

# Step 5: Subset the data to only 20,000 samples (10,000 per class)
class_0_indices = np.where(np.array(labels) == 0)[0][:10000]
class_1_indices = np.where(np.array(labels) == 1)[0][:10000]

# Combine the indices and shuffle them
subset_indices = np.concatenate([class_0_indices, class_1_indices])
np.random.shuffle(subset_indices)

# Subset the data
data_subset = data[subset_indices]
labels_subset = np.array(labels)[subset_indices]

# Step 6: Split the data into train and validation sets (80% train, 20% validation)
train_data, val_data, train_labels, val_labels = train_test_split(
    data_subset, labels_subset, test_size=0.2, random_state=42, stratify=labels_subset
)

# Helper function to save images
def save_images(images, labels, save_dir):
    for idx, (image, label) in enumerate(zip(images, labels)):
        class_folder = os.path.join(save_dir, f"class_{label}")
        img = Image.fromarray(image)
        img.save(os.path.join(class_folder, f"{idx}.png"))

# Step 7: Save images to train and val folders
save_images(train_data, train_labels, train_dir)
save_images(val_data, val_labels, val_dir)

print("Subset of CIFAKE10 dataset distributed into train and validation folders.")

# --- Model and Training Setup ---
from models.densenet_model import create_densenet
from utils.data_loader import create_dataloaders, create_test_loader

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Paths and parameters
data_dir = 'data'
test_split = 0.2  # Use 20% of CIFAR test set for testing

# Load train, validation, and test data
dataloaders = create_dataloaders(data_dir, batch_size=32)
test_loader = create_test_loader(test_split=test_split, batch_size=32)

# Load model
model = create_densenet(num_classes=2).to(device)

# Weighted loss function
from sklearn.utils.class_weight import compute_class_weight
# Compute class weights based on training data
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # Add weight decay

# Use a learning rate scheduler to dynamically reduce the learning rate as the training progresses
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Training function
def train_model(model, criterion, optimizer, dataloaders, num_epochs=15, checkpoint_path="checkpoints"):
    os.makedirs(checkpoint_path, exist_ok=True)
    best_loss = float('inf')

    training_metrics = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": []
    }

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            training_metrics[f'{phase}_loss'].append(epoch_loss)
            training_metrics[f'{phase}_acc'].append(epoch_acc)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Save checkpoint if validation loss improves
            if phase == 'val' and epoch_loss < best_loss:
                scheduler.step(epoch_loss) # Update learning rate based on validation loss
                best_loss = epoch_loss
                torch.save(model.state_dict(), os.path.join(checkpoint_path, f'best_model_epoch_{epoch + 1}.pth'))
                print(f'Checkpoint saved at epoch {epoch + 1}.')

    return model, training_metrics

# Train the model
trained_model, training_metrics = train_model(model, criterion, optimizer, dataloaders, num_epochs=10)

# Save the model
os.makedirs('checkpoints', exist_ok=True)
torch.save(trained_model.state_dict(), 'checkpoints/regFlipSmallerCheckout.pth')
print("Model trained and saved!")

# Evaluate the model
def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Print metrics
    print("Classification Report:")
    print(classification_report(all_labels, all_preds))

    print("Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

# Save metrics
os.makedirs('metrics', exist_ok=True)
metrics_file = 'metrics/regFlipSmaller_metrics.txt'
with open(metrics_file, 'w') as f:
    f.write("Epoch-wise metrics:\n")
    for epoch in range(10):
        f.write(f"Epoch {epoch+1}: Train Loss = {training_metrics['train_loss'][epoch]:.4f}, "
                f"Train Accuracy = {training_metrics['train_acc'][epoch]:.4f}, "
                f"Val Loss = {training_metrics['val_loss'][epoch]:.4f}, "
                f"Val Accuracy = {training_metrics['val_acc'][epoch]:.4f}\n")

# Validation set evaluation
print("Final Evaluation:")
print("Validation Set:")
with open(metrics_file, 'a') as f:
    f.write("\nFinal Evaluation:\nValidation Set:\n")
evaluate_model(trained_model, dataloaders['val'], device)

print("Test Set:")
with open(metrics_file, 'a') as f:
    f.write("\nTest Set:\n")
evaluate_model(trained_model, test_loader, device)

Files already downloaded and verified
Subset of CIFAKE10 dataset distributed into train and validation folders.


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=De

Epoch 1/10
----------


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


train Loss: 0.6769 Acc: 0.5839
val Loss: 0.6728 Acc: 0.6110
Checkpoint saved at epoch 1.
Epoch 2/10
----------
train Loss: 0.6584 Acc: 0.6117
val Loss: 0.6531 Acc: 0.6122
Checkpoint saved at epoch 2.
Epoch 3/10
----------
train Loss: 0.6515 Acc: 0.6182
val Loss: 0.6501 Acc: 0.6232
Checkpoint saved at epoch 3.
Epoch 4/10
----------
train Loss: 0.6374 Acc: 0.6404
val Loss: 0.6219 Acc: 0.6600
Checkpoint saved at epoch 4.
Epoch 5/10
----------


KeyboardInterrupt: 

In [ ]:
!git config --global user.email "huseynovmurad2002@gmail.com"
!git config --global user.name "Murad Hüseynov"

In [ ]:
!git commit -m "regFlipCheckout.pth"

[regFlipCheckout.pth 82335fd] regFlipCheckout.pth
 2 files changed, 16 insertions(+)
 create mode 100644 checkpoints/regFlipSmallerCheckout.pth
 create mode 100644 metrics/regFlipSmaller_metrics.txt


In [ ]:
!git add two_class_classification.ipynb

In [ ]:
!git reset --soft HEAD~1


In [ ]:
!git reset

In [ ]:
!git log

commit 82335fd028e06d4d6df72a4a5f8e422b0bab444f (HEAD -> regFlipCheckout.pth)
Author: Murad Hüseynov <huseynovmurad2002@gmail.com>
Date:   Wed Dec 4 22:30:27 2024 +0000

    regFlipCheckout.pth

commit 1f81eee08116e6a58e5d2b7e5a64a6814f19e5d2 (origin/main, origin/HEAD, main)
Author: Anja Stanic <anja.stanic2001@gmail.com>
Date:   Tue Dec 3 14:36:27 2024 +0000

    Result of inital training for val and test

commit ba1115b84a1f6e4732d0d44ebd00d568cda6f3c7
Author: Murad Huseynov <huseynovmurad2002@gmail.com>
Date:   Mon Dec 2 17:34:36 2024 +0100

    initial model training

commit 96412c460115b73d7d2bd4c503ad405c60a1bb36
Author: Murad Huseynov <huseynovmurad2002@gmail.com>
Date:   Mon Dec 2 14:39:13 2024 +0100

    initial folder structure


In [ ]:
!git add two_class_classification.ipynb

In [ ]:
!git add checkpoints/densenet_ai_vs_authentic.pth

In [ ]:
!git add utils/data_loader.py
!git add utils/evaluation.py
!git add models/densenet_model.py

In [ ]:
!git status

On branch regFlipCheckout.pth
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	cifar-10-batches-py/
	cifar-10-python.tar.gz
	data/train/class_0/
	data/train/class_1/
	data/val/class_0/
	data/val/class_1/
	models/__pycache__/
	utils/__pycache__/

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
!git commit -m "Result of inital training for val and test"

[main 7d673d2d] Result of inital training for val and test
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 checkpoints/densenet_ai_vs_authentic.pth


In [ ]:
!git push origin regFlipCheckout.pth

Enumerating objects: 8, done.
Counting objects: 100% (8/8), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 25.16 MiB | 2.13 MiB/s, done.
Total 6 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
remote: 
remote: Create a pull request for 'regFlipCheckout.pth' on GitHub by visiting:
remote:      https://github.com/Anjaas85/authentic_vs_AI_generated_photos/pull/new/regFlipCheckout.pth
remote: 
To github.com:Anjaas85/authentic_vs_AI_generated_photos.git
 * [new branch]      regFlipCheckout.pth -> regFlipCheckout.pth
